In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [12]:
#マウスリストの作成、神経データが記録されている全フェーズリストの作成
mouse_17list=["ID170704C","ID170704E","ID170704F","ID170704G"]
mouse_13list = ["ID138464-1","ID138464-2","ID138464-3","ID138464-4","ID138464-5","ID138464-6"]
mouse_list=mouse_13list+mouse_17list
phaseall_list=["HC1","B1","B2","HC2","A1","A2","A31","A32","A41","A42","HC3","A5","B3","A6","HC4","A7","B4"]
phase_Ablist=["A1","A2","A31","A32","A41","A42","A5","A6","A7"]
phase_Alist=["A1","A20","A21","A22","A23","A31","A32","A41","A42","A5","A60","A61","A62","A63","A7"]
phase_Blist=["B1","B2","B3","B4"]
phase_Hlist=["HC1","HC2","HC3","HC4"]
phase_ABlist=phase_Ablist+phase_Blist
phase_shocklist=["A2","A6"]
phase_shockdivlist=[0,1,2,3]
phase_daydivlist=[0,3,10,14,17]

In [3]:
#対象マウス決定

for m in range(len(mouse_list)):

    mouse = mouse_list[m]

    #神経データをデータフレームにインポートする
    inputf='%s/python/%s_Longitudinal_Traces.csv' %(mouse,mouse)
    dff=pd.read_csv(inputf,header=0)
    dff0=pd.DataFrame()

    #Acceptのみを抽出したデータフレームの作成
    dff0=dff.iloc[0,:]
    Accept=dff0.index[dff0==' accepted']
    dfa=pd.DataFrame()
    dfa['Time2']=dff.iloc[1:,0]
    for accept_c in range (0,len(Accept)):
        dfa[Accept[accept_c]]=dff[Accept[accept_c]].iloc[1:]
        dfa=dfa.astype('float')

    #Acceptcellのfirst cellとlast cellを抽出
    cell_f=dfa.columns[1]
    cell_l=dfa.columns[dfa.shape[1]-1]

    #レコード間の時間差からphaseを割り当てる
    dfa['dt']=dfa['Time2'].diff()
    dfa['dt'].iloc[0]=0.05
    dfa['phase']=-1
    phase=0
    for i in range(0,dfa.shape[0]):    
        if(dfa['dt'].iloc[i]>10):        
            phase=phase+1
        dfa.iloc[i,dfa.columns.get_loc('phase')]=phaseall_list[phase]

    #割り当てたphaseごとにcsvを生成(XYdataがphaseごとに生成されているため)
    for phase in range(0,len(phaseall_list)):    
        outputf='%s/python/%s_%s_Long_Traces.csv' %(mouse,mouse,phaseall_list[phase])    
        df_phase=dfa[dfa['phase']==phaseall_list[phase]]        
        df_phase.to_csv(outputf,index=False)
        
    #このコード時点でdayごとにstd化してしまう
    for day in range(len(phase_daydivlist)-1):
        
        df_day=pd.DataFrame()
        
        for ph_d in range(phase_daydivlist[day],phase_daydivlist[day+1]):
            
            inputf='%s/python/%s_%s_Long_Traces.csv' %(mouse,mouse,phaseall_list[ph_d])
            
            df_d=pd.read_csv(inputf,header=0)
            
            df_day=pd.concat([df_day,df_d])
            
        for ph_d in range(phase_daydivlist[day],phase_daydivlist[day+1]):            
            
            inputf='%s/python/%s_%s_Long_Traces.csv' %(mouse,mouse,phaseall_list[ph_d])
            
            df_d=pd.read_csv(inputf,header=0)
            
            df_dstd=pd.DataFrame(index=df_d.index,columns=df_d.columns)
            
            for col_b in range(0,df_dstd.columns.get_loc(cell_f)):
                
                    df_dstd[df_dstd.columns[col_b]]=df_d[df_d.columns[col_b]]
            
            for col in range(df_dstd.columns.get_loc(cell_f),df_dstd.columns.get_loc(cell_l)+1):
                
                df_dstd[df_dstd.columns[col]]=(df_d[df_dstd.columns[col]]-df_day[df_dstd.columns[col]].mean())/(df_day[df_dstd.columns[col]].std())

                                                                        
            for col_l in range(df_dstd.columns.get_loc(cell_l)+1,df_dstd.shape[1]):
                
                df_dstd[df_dstd.columns[col_l]]=df_d[df_dstd.columns[col_l]]
                
            outputf='%s/python/%s_%s_Long_Traces_std.csv' %(mouse,mouse,phaseall_list[ph_d])
            
            df_dstd.to_csv(outputf,index=False)
            
                
    #XYデータにヘッダー情報がたくさん入っている、かつcsvなのでヘッダー情報をカラム情報以外取り除き、csvに出力する
    phasepre_list=["B1","B2","A1","A2","A3","A4","A5","B3","A6","A7","B4"]
    for phase_pre in range(0,len(phasepre_list)):
        inputXY='%s/python/%s_XYdata/%s_%s_%s_XY.txt' %(mouse,mouse,mouse,phase_pre+1,phasepre_list[phase_pre])
        outputXY='%s/python/%s_XYdata/%s_%s_XY.csv' %(mouse,mouse,mouse,phasepre_list[phase_pre])
        dfxyp=pd.read_table(inputXY)
        h=dfxyp[dfxyp['TimeFZ4']=='Slice'].index[0]
        dfxy=pd.read_table(inputXY,header=h+1)
        dfxy.to_csv(outputXY,index=False)

    #XYデータがA3,A4を前半後半に分割していないので分割してからcsvに
    phasetransb_list=["A3","A4"]
    phasetransa_list=["A31","A32","A41","A42"]
    for phase_b in range(0,2):
        inputXY='%s/python/%s_XYdata/%s_%s_%s_XY.txt' %(mouse,mouse,mouse,phase_b+5,phasetransb_list[phase_b])
        for phase_a in range(0, 2):
            outputXY='%s/python/%s_XYdata/%s_%s_XY.csv' %(mouse,mouse,mouse,phasetransa_list[2*phase_b+phase_a])
            dfxyp=pd.read_table(inputXY)
            h=dfxyp[dfxyp['TimeFZ4']=='Slice'].index[0]
            dfxy=pd.read_table(inputXY,header=h+1)
            dfxy.iloc[3000*phase_a+0:3000*phase_a+621,:].to_csv(outputXY,index=False)

    #phaseごとに分析をする
    for ph in range(0,len(phase_ABlist)):
        phase=phase_ABlist[ph]
        inputXY='%s/python/%s_XYdata/%s_%s_XY.csv' %(mouse,mouse,mouse,phase)
        dfx = pd.read_csv(inputXY,header=0)
        inputT='%s/python/%s_%s_Long_Traces_std.csv' %(mouse,mouse,phase)
        dfl=pd.read_csv(inputT,header=0)
        
        #ここで神経データと行動データを結合       
        dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))
        for i in range(2,int(dfx.shape[0])):
            for j in range(0,10):
                for k in range(0,len(dfx_m.columns)):
                    dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]
        df=pd.concat([dfx_m, dfl], axis=1)
        df=df.dropna(how='any')
        outputf='%s/python/%s_%s_con_std.csv' %(mouse,mouse,phase)          
        df.to_csv(outputf,index=False)

        #Sliceごとに平均をとったdfmeanの作成
        Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)
        df_mean=pd.DataFrame(index=Index,columns=[])
        for i in range(0,len(df.columns)):
            df_mean[df.columns[i]]=-100
        for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1)):
            i = i_n - int(df.iloc[0].loc['Slice'])
            df_mean.iloc[i].loc['Slice']=i_n
            dfs = df[df['Slice']==i_n]
            for k in range(0,df.columns.get_loc('Time2')+1):
                df_mean.iloc[i,k]=dfs.iloc[0,k]
            for j in range(dfs.columns.get_loc(cell_f),dfs.columns.get_loc(cell_l)+1):
                df_mean.iloc[i,j]=dfs[dfs.columns[j]].mean()
            for l in range(dfs.columns.get_loc(cell_l)+1,dfs.shape[1]):
                df_mean.iloc[i,l]=dfs.iloc[0,l]
        dfmean=df_mean
        outputf='%s/python/%s_%s_mean_std.csv' %(mouse,mouse,phase)          
        dfmean.to_csv(outputf,index=False)
        
    for ph in range(len(phase_Hlist)):
        phase=phase_Hlist[ph]
        inputT='%s/python/%s_%s_Long_Traces_std.csv' %(mouse,mouse,phase)
        dfl=pd.read_csv(inputT,header=0)
        
        #ここで神経データと行動データを結合       
        dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))
        for i in range(2,int(dfx.shape[0])):
            for j in range(0,10):
                for k in range(0,len(dfx_m.columns)):
                    dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]
        df=dfl
        df=df.dropna(how='any')
        outputf='%s/python/%s_%s_con_std.csv' %(mouse,mouse,phase)          
        df.to_csv(outputf,index=False)

        #Sliceごとに平均をとったdfmeanの作成
        Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)
        df_mean=pd.DataFrame(index=Index,columns=[])
        for i in range(0,len(df.columns)):
            df_mean[df.columns[i]]=-100
        for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1)):
            i = i_n - int(df.iloc[0].loc['Slice'])
            df_mean.iloc[i].loc['Slice']=i_n
            dfs = df[df['Slice']==i_n]
            for k in range(0,df.columns.get_loc('Time2')+1):
                df_mean.iloc[i,k]=dfs.iloc[0,k]
            for j in range(dfs.columns.get_loc(cell_f),dfs.columns.get_loc(cell_l)+1):
                df_mean.iloc[i,j]=dfs[dfs.columns[j]].mean()
            for l in range(dfs.columns.get_loc(cell_l)+1,dfs.shape[1]):
                df_mean.iloc[i,l]=dfs.iloc[0,l]
        dfmean=df_mean
        outputf='%s/python/%s_%s_mean_std.csv' %(mouse,mouse,phase)          
        dfmean.to_csv(outputf,index=False)

/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243

In [4]:
for m in range(len(mouse_list)):
    
    mouse=mouse_list[m]
    
    for ph_p in range(len(phase_shocklist)):
    
        phase_p=phase_shocklist[ph_p]
        
        inputf='%s/python/%s_%s_mean_std.csv'%(mouse,mouse,phase_p)
    
        dfmean_p=pd.read_csv(inputf,header=0)
    
        dfmean_p['session']=dfmean_p['Shock']-dfmean_p['Shock'].shift(1)
    
        dfmean_p['session'].iloc[0]=1
    
        dfmean_p['session'].iloc[dfmean_p.shape[0]-1]=1
    
        division_list=dfmean_p[dfmean_p['session']==1].index
    
        for ph in range(len(phase_shockdivlist)):
        
            df_shock=dfmean_p.iloc[division_list[ph]:division_list[ph+1],:]

            outputf='%s/python/%s_%s%s_mean_std.csv'%(mouse,mouse,phase_p,phase_shockdivlist[ph])
        
            df_shock.to_csv(outputf,index=False)

/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
for m in range(len(mouse_list)):
    
    mouse=mouse_list[m]
    
    df_all=pd.DataFrame()
    
    for ph in range(len(phase_Alist)):
        
        phase=phase_Alist[ph]
        
        df_phase=pd.DataFrame()
        
        inputf='%s/python/%s_%s_mean_std.csv'%(mouse,mouse,phase)
        
        df_phase=pd.read_csv(inputf,header=0)
        
        df_all=pd.concat([df_all,df_phase])
        
    outpuf='%s/python/%s_mean_std_Aall.csv'%(mouse,mouse)
        
    df_all.to_csv(outpuf,index=False)

/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [6]:
for m in range(len(mouse_list)):
    
    mouse=mouse_list[m]
    
    df_all=pd.DataFrame()
    
    for ph in range(len(phase_Blist)):
        
        phase=phase_Blist[ph]
        
        df_phase=pd.DataFrame()
        
        inputf='%s/python/%s_%s_mean_std.csv'%(mouse,mouse,phase)
        
        df_phase=pd.read_csv(inputf,header=0)
        
        df_all=pd.concat([df_all,df_phase])
        
    outpuf='%s/python/%s_mean_std_Ball.csv'%(mouse,mouse)
        
    df_all.to_csv(outpuf,index=False)

In [13]:
for m in range(len(mouse_list)):
    
    mouse = mouse_list[m]
    
    #神経データをデータフレームにインポートする
    inputf='%s/python/%s_Longitudinal_Traces.csv' %(mouse,mouse)
    dff=pd.read_csv(inputf,header=0)
    dff0=pd.DataFrame()

    #Acceptのみを抽出したデータフレームの作成
    dff0=dff.iloc[0,:]
    Accept=dff0.index[dff0==' accepted']
    dfa=pd.DataFrame()
    dfa['Time2']=dff.iloc[1:,0]
    for accept_c in range (0,len(Accept)):
        dfa[Accept[accept_c]]=dff[Accept[accept_c]].iloc[1:]
        dfa=dfa.astype('float')

    #Acceptcellのfirst cellとlast cellを抽出
    cell_f=dfa.columns[1]
    cell_l=dfa.columns[dfa.shape[1]-1]
    
    for ph in range(len(phase_Hlist)):
        phase=phase_Hlist[ph]
        inputT='%s/python/%s_%s_Long_Traces_std.csv' %(mouse,mouse,phase)
        dfl=pd.read_csv(inputT,header=0)
        
        #ここで神経データと行動データを結合       
        df=dfl
        df['Slice']=df.index//10
        df=df.dropna(how='any')
        outputf='%s/python/%s_%s_con_std.csv' %(mouse,mouse,phase)          
        df.to_csv(outputf,index=False)

        #Sliceごとに平均をとったdfmeanの作成
        Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)
        df_mean=pd.DataFrame(index=Index,columns=[])
        for i in range(0,len(df.columns)):
            df_mean[df.columns[i]]=-100
        for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1)):
            i = i_n - int(df.iloc[0].loc['Slice'])
            df_mean.iloc[i].loc['Slice']=i_n
            dfs = df[df['Slice']==i_n]
            for k in range(0,df.columns.get_loc('Time2')+1):
                df_mean.iloc[i,k]=dfs.iloc[0,k]
            for j in range(dfs.columns.get_loc(cell_f),dfs.columns.get_loc(cell_l)+1):
                df_mean.iloc[i,j]=dfs[dfs.columns[j]].mean()
            for l in range(dfs.columns.get_loc(cell_l)+1,dfs.shape[1]):
                df_mean.iloc[i,l]=dfs.iloc[0,l]
        dfmean=df_mean
        outputf='%s/python/%s_%s_mean_std.csv' %(mouse,mouse,phase)          
        dfmean.to_csv(outputf,index=False)

/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is

/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rentakakura/anaconda/envs/jupyter-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,

In [15]:
for m in range(len(mouse_list)):
    
    mouse=mouse_list[m]
    
    df_all=pd.DataFrame()
    
    for ph in range(len(phase_Hlist)):
        
        phase=phase_Hlist[ph]
        
        df_phase=pd.DataFrame()
        
        inputf='%s/python/%s_%s_mean_std.csv'%(mouse,mouse,phase)
        
        df_phase=pd.read_csv(inputf,header=0)
        
        df_all=pd.concat([df_all,df_phase])
        
    outpuf='%s/python/%s_mean_std_HCall.csv'%(mouse,mouse)
        
    df_all.to_csv(outpuf,index=False)